In [1]:
import os

In [2]:
%pwd

'/Users/vivek/work/GIT/Triluxo/reserch'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/vivek/work/GIT/Triluxo'

In [5]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [6]:
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
index_name = "triluxo"


In [8]:
api_key = os.getenv("PINECONE")

In [9]:
import os
from pinecone import Pinecone, ServerlessSpec


# index_name = "hybrid-search-pinecone"

#initizing the pinecone clinet
pc = Pinecone(api_key=api_key)

try:
    # Retrieve the list of existing indexes
    existing_indexes = pc.list_indexes()
    print(f"Existing indexes: {existing_indexes}")

    if index_name not in existing_indexes:
        # Create the index if it doesn't exist
        pc.create_index(
            name=index_name,
            dimension=384,  # Dimension of your dense model
            metric='dotproduct',  # Metric for similarity search
            spec=ServerlessSpec(cloud='aws', region='us-east-1')
        )
        print(f"Index '{index_name}' created successfully.")
    else:
        print(f"Index '{index_name}' already exists.")

except Exception as e:
    if e.status == 409:
        # Handle the case where the index already exists
        print(f"Index '{index_name}' already exists. Proceeding to use it.")
    else:
        # Handle other potential exceptions
        print(f"An error occurred while creating the index: {e}")

/opt/miniconda3/envs/triluxo/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


Existing indexes: [{
    "name": "hybrid-search-pinecone",
    "dimension": 384,
    "metric": "dotproduct",
    "host": "hybrid-search-pinecone-nqxn8hs.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "deletion_protection": "disabled"
}]
Index 'triluxo' created successfully.


In [10]:
index = pc.Index(index_name)
index

In [11]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")


In [12]:
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")

In [13]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [14]:
from pinecone_text.sparse import BM25Encoder

bm25_encoder = BM25Encoder().default() #defining the default encoder i.e .default()
bm25_encoder

In [15]:
# Function to convert text file to Python list
def file_to_list(filename):
    with open("output.txt", 'r') as file:
        # Read the file content
        content = file.read()
        # Split the content by lines and return as a list
        lines_list = content.splitlines()  # Each line will be a separate item in the list
    return lines_list

# Example usage
filename = 'example.txt'  # Replace with your text file name
result_list = file_to_list(filename)
print(result_list)

['Brainlox: Learn technical courses.', 'Courses ', 'Technical', 'Academic', 'Language', 'Music', 'Lifestyle', 'Book a Free Demo Now', 'Sign In', 'FAQ', 'Contact Us', 'Practice Python', 'Learn Now', 'Home', 'Courses', 'Courses', 'We found great courses available for you', '$', '20', 'per session', 'LEARN SCRATCH PROGRAMING', '', 'Scratch Course is the foundation of coding and is a building block of a coding journey. If you want', ' ', '16', ' Lessons', 'View Details', '$', '30', 'per session', 'LEARN CLOUD COMPUTING BASICS-AWS', '', 'In this course we are going to cover the basics and the most important services on AWS,', 'At the end ', ' ', '20', ' Lessons', 'View Details', '$', '20', 'per session', 'LEARN MOBILE DEVELOPMENT', '', 'Mobile application development is the process of creating software applications that run on a mobil', ' ', '24', ' Lessons', 'View Details', '$', '20', 'per session', 'LEARN CORE JAVA PROGRAMMING ONLINE', '', 'Java is a very popular high-level, class-based, 

In [16]:
def chunk_list(data, chunk_size, overlap):
    """
    Split a list into chunks with specified chunk size and overlap length.

    :param data: List to be chunked
    :param chunk_size: Size of each chunk
    :param overlap: Overlap length between consecutive chunks
    :return: List of chunks
    """
    if chunk_size <= overlap:
        raise ValueError("Chunk size must be greater than overlap length.")
    
    # Create chunks with overlap
    chunks = []
    for i in range(0, len(data), chunk_size - overlap):
        chunk = data[i:i + chunk_size]
        chunks.append(chunk)
        if len(chunk) < chunk_size:
            break  # Stop if the last chunk is smaller than the chunk size
    return chunks

In [17]:
chunk = chunk_list(result_list, 50, 20)

In [18]:
chunk_data = ["".join(chunk[i]) for i in range(len(chunk))]

In [19]:
chunk_data

['Brainlox: Learn technical courses.Courses TechnicalAcademicLanguageMusicLifestyleBook a Free Demo NowSign InFAQContact UsPractice PythonLearn NowHomeCoursesCoursesWe found great courses available for you$20per sessionLEARN SCRATCH PROGRAMINGScratch Course is the foundation of coding and is a building block of a coding journey. If you want 16 LessonsView Details$30per sessionLEARN CLOUD COMPUTING BASICS-AWSIn this course we are going to cover the basics and the most important services on AWS,At the end  20 LessonsView Details$20per sessionLEARN MOBILE DEVELOPMENTMobile application development is the process of creating software applications that run on a mobil 24 LessonsView Details$20',
 'LEARN CLOUD COMPUTING BASICS-AWSIn this course we are going to cover the basics and the most important services on AWS,At the end  20 LessonsView Details$20per sessionLEARN MOBILE DEVELOPMENTMobile application development is the process of creating software applications that run on a mobil 24 Lesson

In [20]:
#resolving the nltk error 
import nltk

# Download the punkt tokenizer model
nltk.download('punkt_tab')




[nltk_data] Downloading package punkt_tab to /Users/vivek/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [21]:
#tfidf values  on these sentences
bm25_encoder.fit(chunk_data)


100%|██████████| 22/22 [00:00<00:00, 603.35it/s]


In [22]:
#store the values as json files
bm25_encoder.dump("bm25_values.json")

#loading the encoder object
bm25_loaded = BM25Encoder().load("bm25_values.json")

In [23]:
#creating a retriver object
retriver = PineconeHybridSearchRetriever(embeddings=embeddings, sparse_encoder=bm25_encoder, index=index)

In [24]:
retriver

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x14a8f2750>, index=<pinecone.data.index.Index object at 0x10d703450>)

In [27]:
retriver.add_texts(chunk_data)

100%|██████████| 1/1 [00:03<00:00,  3.33s/it]


In [28]:
retriver.invoke("7 LessonsView Details$22per sessionThe AI Writer")

[Document(metadata={'score': 0.575161457}, page_content=' 7 LessonsView Details$22per sessionThe AI Writer\'s Masterclass: Innovation and Inspiration in Creative Writing! (For Kids)Enhance your creative writing skills with AI! Join our 10-day course and explore AI\'s role in writin 10 LessonsView Details$30per sessionWeb Development Pro: Intermediate LevelReady to unlock the full potential of web development? Join our "Intermediate Web Development Master 8 LessonsView Details$20per sessionScratch Playground: Create a Scroller Game!Scratch Playground : Create a Scroller Game is an intermediate level course designed to teach kids t 8 LessonsView Details$25per sessionAI Pro: Creative Writing Camp for Adults🚀 Welcome to Creative Writing with AI for ADULTS! Unleash your creativity with cutting-edge AI tec 10 LessonsView Details$20per sessionPython Playground: Create Your Own Snake GameThrough this course, participants will learn the fundamentals of Python programming language, as wel 8 Less